In [1]:
import pandas as pd
import csv
import os


Dado el gran volumen de datos y acorde a la documentación de los datos en [kaggle](https://www.kaggle.com/datasets/netflix-inc/netflix-prize-data) *"CustomerIDs range from 1 to 2649429, with gaps. There are 480189 users."*, se decide limitar la cantidad de usuarios a usar dado un id máximo. En este caso, se usará el id 100000 como límite superior.

In [2]:

def extract_df(path,max_id=100000):
    """
    Extrae y filtra los datos de un archivo CSV de Netflix Prize.

    Parameters
    ----------
    path : str
        Ruta al archivo CSV.
    max_id : int
        Valor máximo de customer_id a extraer.

    Returns
    -------
    pandas.DataFrame
        DataFrame con las columnas 'movie_id', 'customer_id', 'rating' y 'date'.
    """

    with open(path, newline='') as csvfile:
        reader = csv.reader(csvfile)
        
        data = []

        for row in reader:

            if row[0].endswith(':'):
                # dada la estructura del archivo, la primera columna termina con ':'
                movie_id = row[0][:-1]
                
            else:
                # las siguientes filas contienen los datos de las películas
                customer_id = row[0]  
                rating = row[1]  
                date = row[2]  

                # solo tomamos datos de los customer_id menores o iguales a max_id
                if int(customer_id) <= max_id:
                    data.append((movie_id, customer_id, rating, date))

    df = pd.DataFrame(data, columns=['movie_id', 'customer_id', 'rating', 'date'])
    return df


# carpeta con la información de Netflix Prize
data_path = "Netflix_Prize_data"
# listar los archivos de datos de entrenamiento
files = [f for f in os.listdir(data_path)  if f.startswith('combined_data')] 

# concatenar los datos de los archivos de entrenamiento
df = pd.concat([extract_df(os.path.join(data_path, f)) for f in files], ignore_index=True)
# ordenar los datos por customer_id, fecha y rating
df.sort_values(by=['customer_id',"date","rating"],ascending=[True,True,False], inplace=True)



De esta manera podemos tener un dataset más manejable y que permita realizar pruebas de manera más rápida.

In [3]:
# convierte la columna rating a entero
df.rating = df.rating.astype(int)


Dado que no contamos con más detalle temporal que el día, se decide asignar como siguiente película a recomendar la película con mayor rating vista en el día, para los registros de misma película y misma recomendación se descartan. 

In [4]:
# ids de películas con mejor rating vistas en el día por cada cliente
idx = df.groupby(['customer_id', 'date'])['rating'].idxmax()

# df con los registros de las películas con mejor rating vistas en el día por cada cliente
result = df.loc[idx]

# filtro solo columnas para cruzar
result = result[["customer_id","date","movie_id","rating"]].rename(columns={"movie_id":"best_movie_id","rating":"best_rating"})

# Cruce de la información
df = pd.merge(df,result, on=["customer_id","date"], how="left")


df.drop(columns=["date"],inplace=True)

# solo se toman registros donde la película vista es diferente a la mejor película vista en el día (la cual se toma como recomendación)
df[df.movie_id != df.best_movie_id]

,movie_id,customer_id,rating,date,best_movie_id
1,14367,10,5,2002-12-20,9340
2,571,10,4,2002-12-20,9340
3,2122,10,4,2002-12-20,9340
4,6972,10,4,2002-12-20,9340
5,15124,10,4,2002-12-20,9340
...,...,...,...,...,...
3737134,10168,99994,2,2005-10-16,4996
3737135,11149,99994,2,2005-10-16,4996
3737136,13050,99994,2,2005-10-16,4996
3737137,13081,99994,2,2005-10-16,4996


In [5]:
# muestra los primeros 15 registros del customer_id 10
df[df.customer_id == '10'].head(15)

,movie_id,customer_id,rating,date,best_movie_id
0,9340,10,5,2002-12-20,9340
1,14367,10,5,2002-12-20,9340
2,571,10,4,2002-12-20,9340
3,2122,10,4,2002-12-20,9340
4,6972,10,4,2002-12-20,9340
5,15124,10,4,2002-12-20,9340
6,6029,10,3,2002-12-20,9340
7,11064,10,3,2002-12-20,9340
8,16439,10,3,2002-12-20,9340
9,17308,10,3,2002-12-20,9340


In [6]:
# guardado de los datos en formato parquet dentro de la carpeta data_path
df.to_parquet(data_path+'/netflix_data_sample.parquet', index=False)